## Process data

In [162]:
import load
from scipy.io import loadmat, savemat

images, labels = load.load_CIFAR10()


Loading CIFAR10...
----------------
Elapsed: 0.44 sec


In [163]:
print(images.shape, images.dtype)
print(labels.shape, labels.dtype)
#savemat("./saved/raw/CIFAR10.mat", mdict={'X':images, 'Y':labels})

(60000, 32, 32, 3) uint8
(60000,) int32


In [164]:
images_norm = rescale(images)


Normalizing data...
-------------------
Elapsed: 1.49 sec


In [165]:
print(images_norm.shape, images_norm.dtype)
#savemat('./saved/rescaled/CIFAR10.mat', mdict={'X':images_norm, 'Y':labels})

(60000, 32, 32, 3) float32


## SSC Comparison (performance2)

In [1]:
import load
images_yaleb, labels_yaleb = load.load_YaleB()
images_coil, labels_coil = load.load_Coil20()
images_mnist, labels_mnist = load.load_MNIST()
images_cifar, labels_cifar = load.load_CIFAR10()


Loading YaleB...
----------------
Elapsed: 9.52 sec

Loading Coil20...
----------------
Elapsed: 8.37 sec

Loading MNIST...
----------------
Elapsed: 5.65 sec

Loading CIFAR10...
----------------
Elapsed: 4.89 sec


In [ ]:
k = len(np.unique(labels))
alpha = 20.0
maxIter = 63

In [ ]:
if(not trainC):
    savemat('./temp.mat', mdict={'X': images_HM2})
else:
    savemat('./temp.mat', mdict={'C': C})
grps = eng.SSC_modified(k, 0, False, alpha, False, 1, 1e-20, maxIter, True, 0, trainC)
C_after = loadmat("./temp.mat")['C']
labels_pred = np.asarray(grps, dtype=np.int32).flatten()

In [ ]:
from supporting_files.ji_zhang import err_rate
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_rand_score as ari

predicted = labels_pred
print("Accuracy: ", str(1-err_rate(labels, predicted)))
print("NMI: ", str(nmi(labels, predicted, average_method="geometric")))
print("ARI: ", str(ari(labels, predicted)))

## Process optimization visualization

In [31]:
from figures.hyper_opt import *

visual_save("optims/scenario21/forest_0_100.opt", N=8)

C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\scipy\io\matlab\miobase.py:414: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  arr[empties] = ' '
